This Notebook can be viewed at the following link: https://colab.research.google.com/drive/15IUTc-BUjwehb2FdDJq44-64zN66KKy9?usp=sharing

Authors:

* Stefano Ribes, ribes.stefano@gmail.com | stefano.ribes@gaisler.com
* Alessandro Palumbo, alessandro.palumbo@uniroma2.it

# HTH Detection and Classification

## Setup

In [ ]:
import os

data_dir = os.path.join(os.path.abspath(''), '..', 'data')
models_dir = os.path.join(data_dir, 'models')
fig_dir = os.path.join(data_dir, 'figures')

# if not os.path.exists(data_dir):
#     data_dir = os.path.join('/', 'tmp')
#     !gdown https://drive.google.com/file/d/1gKBc8CyZnC0bYl9WzKEFc4e2BhL4lvU0/view?usp=sharing -O /tmp/triggered.csv
#     !gdown https://drive.google.com/file/d/1gKBc8CyZnC0bYl9WzKEFc4e2BhL4lvU0/view?usp=sharing -O /tmp/not_triggered.csv
#     !cat /tmp/triggered.csv

data_dir

## Data Cleaning

In [ ]:
import pandas as pd
import os

triggered_df = pd.read_csv(os.path.join(data_dir, 'raw', 'triggered.csv'))
not_triggered_df = pd.read_csv(os.path.join(data_dir, 'raw', 'not_triggered.csv'))

triggered_df.describe().round(2)

In [ ]:
not_triggered_df.describe().round(2)

In [ ]:
old2new = {
    'benchmark': 'benchmark',
    '% dyn pow': 'avg_dynamic_power',
    'power': 'avg_static_power',
    'FF': 'FF',
    'LUT': 'LUT',
    'timing': 'worst_negative_slack',
    'cycles': 'num_cycles',
    'instr ret': 'num_instructions_retired',
    'LSU': 'datamem_wait_cycles',
    'fetch wait': 'fetch_wait_cycles',
    'load': 'num_load_instructions',
    'store': 'num_store_instructions',
    'jump': 'num_jump_instructions',
    'cond br': 'num_conditional_branch_instructions',
    'tak c bran': 'num_taken_conditional_branch_instructions',
    'compr ins': 'num_compressed_instructions',
    'mult wait': 'mult_wait_cycles',
    'divd wait': 'divd_wait_cycles',
    'Temp': 'temperature',
    'target': 'ht_type',
}
# Rename HT classes
ht_types = {
    0: 'free',
    1: 'mitm',
    2: 'fetch',
    3: 'critical',
    4: 'clk_mod',
}
# Rename dataframes columns
triggered_df = triggered_df.rename(columns=old2new).replace({'ht_type': ht_types})
not_triggered_df = not_triggered_df.rename(columns=old2new).replace({'ht_type': ht_types})
triggered_df.head()

In [ ]:
ax = triggered_df.plot.hist(by='avg_dynamic_power', bins=10, alpha=0.9)
ax.plot()



### Reading from file

Target HTH classes: `golden`, `instr`, `fetch`, `critical` and `clk mod`.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

hth_classes = ['golden', 'instr', 'fetch', 'critical', 'clk_mod']
hth_dict = {
    0: 'golden',
    1: 'instr',
    2: 'fetch',
    3: 'critical',
    4: 'clk_mod'
}

def get_dataset(triggered_file, not_triggered_file=None, convert_labels=False):
    # Read the CSV file of data
    data = pd.read_csv(triggered_file).dropna()
    data['triggered'] = 1
    # Merge two files if required
    if not_triggered_file is not None:
        data2 = pd.read_csv(not_triggered_file).dropna()
        data2['triggered'] = 0
        data = pd.concat([data, data2])
    # Convert numerical labels to string
    if convert_labels:
        for hth_id in hth_dict.keys():
            data.loc[data['target'] == hth_id, 'target'] = hth_dict[hth_id]
    # Shuffle the dataset
    return data.sample(frac=1.0, random_state=1)

def split_dataset(data_shuffled, train_perc=0.8):
    # Splitting into train and test datasets
    train_len = int(len(data_shuffled) * train_perc)
    data_shuffled_train = data_shuffled.iloc[:train_len, :]
    data_shuffled_test = data_shuffled.iloc[train_len:, :]
    # Split into input part X and output part Y
    # Train
    Xtrain = data_shuffled_train.drop(['target', 'triggered'], axis=1)
    Ytrain = data_shuffled_train['target'].to_frame()
    # Test
    Xtest = data_shuffled_test.drop(['target', 'triggered'], axis=1)
    Ytest = data_shuffled_test['target'].to_frame()
    return Xtrain, Ytrain, Xtest, Ytest

triggered = os.path.join(data_dir, 'raw', 'triggered.csv')
not_triggered = os.path.join(data_dir, 'raw', 'not_triggered.csv')
df = get_dataset(triggered, not_triggered)

Xtrain8020, Ytrain8020, Xtest8020, Ytest8020 = split_dataset(df, train_perc=0.8)
Xtrain6535, Ytrain6535, Xtest6535, Ytest6535 = split_dataset(df, train_perc=0.65)
Xtrain, Ytrain, Xtest, Ytest = split_dataset(df)
Xtrain

In [ ]:
len(Xtrain)

In [ ]:
df_triggered = get_dataset(triggered, convert_labels=False)
df_not_triggered = get_dataset(not_triggered, convert_labels=False)
Xtrain8020_trig, Ytrain8020_trig, Xtest8020_trig, Ytest8020_trig = split_dataset(df_triggered, train_perc=0.8)
Xtrain6535_trig, Ytrain6535_trig, Xtest6535_trig, Ytest6535_trig = split_dataset(df_triggered, train_perc=0.65)

Xtrain8020_ntrig, Ytrain8020_ntrig, Xtest8020_ntrig, Ytest8020_ntrig = split_dataset(df_not_triggered, train_perc=0.8)
Xtrain6535_ntrig, Ytrain6535_ntrig, Xtest6535_ntrig, Ytest6535_ntrig = split_dataset(df_not_triggered, train_perc=0.65)

Xtrain8020_trig

In [ ]:
print(f'len(Xtrain8020_trig):  {len(Xtrain8020_trig)}')
print(f'len(Xtrain6535_trig):  {len(Xtrain6535_trig)}')
print(f'len(Xtrain8020_ntrig): {len(Xtrain8020_ntrig)}')
print(f'len(Xtrain6535_ntrig): {len(Xtrain6535_ntrig)}')
print(f'')
print(f'len(Xtest8020_trig):  {len(Xtest8020_trig)}')
print(f'len(Xtest6535_trig):  {len(Xtest6535_trig)}')
print(f'len(Xtest8020_ntrig): {len(Xtest8020_ntrig)}')
print(f'len(Xtest6535_ntrig): {len(Xtest6535_ntrig)}')

### Labels Distributions

Transform numerical labels (dataframes) into string labels (array of strings) (useful for plotting and inspection).

In [ ]:
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler

def preprocess_labels(Y, binary_classification=False):
    if type(Y) is np.ndarray:
        return Y
    # Dictionarize labels
    dv = DictVectorizer(sparse=False, separator='')
    Y_encoded = dv.fit_transform(Y.to_dict('records'))
    # Apply labels
    le = LabelEncoder()
    if binary_classification:
        le.fit(['no_hth', 'with_hth'])
    else:
        le.fit(hth_classes)
    return le.inverse_transform(Y_encoded.ravel().astype(int))

Ytrain8020 = preprocess_labels(Ytrain8020)
Ytest8020 = preprocess_labels(Ytest8020)
Ytrain6535 = preprocess_labels(Ytrain6535)
Ytest6535 = preprocess_labels(Ytest6535)

Ytrain8020_trig = preprocess_labels(Ytrain8020_trig)
Ytest8020_trig = preprocess_labels(Ytest8020_trig)
Ytrain6535_trig = preprocess_labels(Ytrain6535_trig)
Ytest6535_trig = preprocess_labels(Ytest6535_trig)

Ytrain8020_ntrig = preprocess_labels(Ytrain8020_ntrig)
Ytest8020_ntrig = preprocess_labels(Ytest8020_ntrig)
Ytrain6535_ntrig = preprocess_labels(Ytrain6535_ntrig)
Ytest6535_ntrig = preprocess_labels(Ytest6535_ntrig)
Ytest8020


Ideally, since we have 5 classes, each class should appear around 20% in each dataset. The following graphs investigate that.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

raw_data = {
    'hth_classes': [],
    'counts': [],
    'categories': []
}

for hth_type in hth_classes:
    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytrain8020 == hth_type) / len(Ytrain8020)]
    raw_data['categories'] += ['Ytrain']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytest8020 == hth_type) / len(Ytest8020)]
    raw_data['categories'] += ['Ytest']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytrain8020_trig == hth_type) / len(Ytrain8020_trig)]
    raw_data['categories'] += ['Ytrain (triggered)']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytest8020_trig == hth_type) / len(Ytest8020_trig)]
    raw_data['categories'] += ['Ytest  (triggered)']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytrain8020_ntrig == hth_type) / len(Ytrain8020_ntrig)]
    raw_data['categories'] += ['Ytrain (non-triggered)']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytest8020_ntrig == hth_type) / len(Ytest8020_ntrig)]
    raw_data['categories'] += ['Ytest  (non-triggered)']

ax = sns.barplot(x='hth_classes', y='counts', hue='categories', data=raw_data)

ax.grid(which='both', axis='y', alpha=0.7, zorder=1)
plt.ylabel('Class Counts [%]')
plt.xlabel('Class Labels')
plt.title('HTH classes counts in 80%-20% datasets')
# plt.tight_layout()
plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left')
plt.show()

In [ ]:
raw_data = {
    'hth_classes': [],
    'counts': [],
    'categories': []
}

for hth_type in hth_classes:
    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytrain6535 == hth_type) / len(Ytrain6535)]
    raw_data['categories'] += ['Ytrain']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytest6535 == hth_type) / len(Ytest6535)]
    raw_data['categories'] += ['Ytest']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytrain6535_trig == hth_type) / len(Ytrain6535_trig)]
    raw_data['categories'] += ['Ytrain (triggered)']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytest6535_trig == hth_type) / len(Ytest6535_trig)]
    raw_data['categories'] += ['Ytest  (triggered)']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytrain6535_ntrig == hth_type) / len(Ytrain6535_ntrig)]
    raw_data['categories'] += ['Ytrain (non-triggered)']

    raw_data['hth_classes'] += [hth_type]
    raw_data['counts'] += [sum(Ytest6535_ntrig == hth_type) / len(Ytest6535_ntrig)]
    raw_data['categories'] += ['Ytest  (non-triggered)']

ax = sns.barplot(x='hth_classes', y='counts', hue='categories', data=raw_data)

ax.grid(which='both', axis='y', alpha=0.7, zorder=1)
plt.ylabel('Class Counts [%]')
plt.xlabel('Class Labels')
plt.title('HTH classes counts in 65%-35% datasets')
# plt.tight_layout()
plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left')
plt.show()

### Dispersion Diagrams

In [ ]:
triggered = os.path.join(data_dir, 'triggered.csv')
not_triggered = os.path.join(data_dir, 'not_triggered.csv')
df = get_dataset(triggered, not_triggered, convert_labels=True)
df['Samples ID'] = df.index

for feature in ['timing', 'LUT', 'FF', 'Temp']:
    sns.scatterplot(data=df, x='Samples ID', y=feature, hue='target')
    plt.title(f'Dispersion Diagram for feature "{feature}"')
    plt.grid(which='both', axis='both', alpha=0.7, zorder=1)
    plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left')
    plt.show()

In [ ]:
triggered = os.path.join(data_dir, 'triggered.csv')
not_triggered = os.path.join(data_dir, 'not_triggered.csv')
df = get_dataset(triggered, not_triggered, convert_labels=True)
df['Samples ID'] = df.index

for feature in ['cycles', 'instr ret', 'LSU', 'fetch wait', 'load',
                        'store', 'jump', 'cond br', 'tak c bran', 'compr ins',
                        'mult wait', 'divd wait', 'benchmark', ]:
    sns.scatterplot(data=df, x='Samples ID', y=feature, hue='target')
    plt.title(f'Dispersion Diagram for feature "{feature}"')
    plt.grid(which='both', axis='both', alpha=0.7, zorder=1)
    plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left')
    plt.show()

In [ ]:
triggered = os.path.join(data_dir, 'triggered.csv')
not_triggered = os.path.join(data_dir, 'not_triggered.csv')
df = get_dataset(triggered, not_triggered, convert_labels=True)
df['Samples ID'] = df.index

df = df[df['cycles'] < 100000]
df = df.rename(lambda x: 'Timing [ns]' if x == 'timing' else x, axis='columns')
df = df.rename(lambda x: 'Clock Cycles' if x == 'cycles' else x, axis='columns')

ax = sns.scatterplot(data=df, y='Timing [ns]', x='Clock Cycles', hue='target')

ax.axhline(7.65, ls='--', c='C0')
ax.axvline(11500, ls='--', c='C1')

# plt.title(f'Dispersion Diagram for feature "{feature}"')
plt.grid(which='both', axis='both', alpha=0.7, zorder=1)
# plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left')
plt.legend(loc='upper right')
plt.savefig(os.path.join(fig_dir, f'pf+if_vs_if.pdf'), bbox_inches='tight')
plt.show()

### Box Plots

In [ ]:
triggered = os.path.join(data_dir, 'triggered.csv')
not_triggered = os.path.join(data_dir, 'not_triggered.csv')
df = get_dataset(triggered, not_triggered, convert_labels=True)

for feature in ['timing', 'LUT', 'FF', 'Temp']:
    sns.boxplot(x='target', y=feature, data=df)
    plt.title(f'Box plot for feature "{feature}"')
    plt.grid(which='both', axis='both', alpha=0.7, zorder=1)
    plt.show()

### Features Correlation

In [ ]:
performance_features = ['cycles', 'instr ret', 'LSU', 'fetch wait', 'load',
                        'store', 'jump', 'cond br', 'tak c bran', 'compr ins',
                        'mult wait', 'divd wait', 'benchmark', ]
synthesis_features = ['FF', 'LUT', 'timing', '% dyn pow', 'power', 'Temp', ]

triggered = os.path.join(data_dir, 'triggered.csv')
not_triggered = os.path.join(data_dir, 'not_triggered.csv')
df = get_dataset(triggered, not_triggered, convert_labels=True)
df = df.drop(['triggered'], axis=1)
# Order the columns for better distinguishing the two feature sets
df = df[performance_features + synthesis_features]

corr = df.corr()
plt.figure(figsize=(16, 8))
sns.heatmap(corr, cmap='vlag', annot=True)
plt.show()

### Remove Synthesis Features

In [ ]:
df = get_dataset(triggered, not_triggered).drop(synthesis_features, axis=1)

Xtrain8020_nosyn, Ytrain8020_nosyn, Xtest8020_nosyn, Ytest8020_nosyn = split_dataset(df, train_perc=0.8)
Xtrain6535_nosyn, Ytrain6535_nosyn, Xtest6535_nosyn, Ytest6535_nosyn = split_dataset(df, train_perc=0.65)

Ytrain8020_nosyn = preprocess_labels(Ytrain8020_nosyn)
Ytest8020_nosyn = preprocess_labels(Ytest8020_nosyn)
Ytrain6535_nosyn = preprocess_labels(Ytrain6535_nosyn)
Ytest6535_nosyn = preprocess_labels(Ytest6535_nosyn)

Ytest8020_nosyn

In [ ]:
print(f'len(Xtrain8020_nosyn): {len(Xtrain8020_nosyn)}')
print(f'len(Xtrain8020_trig):  {len(Xtrain8020_trig)}')
print(f'len(Xtrain8020_ntrig): {len(Xtrain8020_ntrig)}')
print(f'len(Xtrain6535_nosyn): {len(Xtrain6535_nosyn)}')
print(f'len(Xtrain6535_trig):  {len(Xtrain6535_trig)}')
print(f'len(Xtrain6535_ntrig): {len(Xtrain6535_ntrig)}')
print('')
print(f'len(Xtest8020_nosyn): {len(Xtest8020_nosyn)}')
print(f'len(Xtest8020_trig):  {len(Xtest8020_trig)}')
print(f'len(Xtest8020_ntrig): {len(Xtest8020_ntrig)}')
print(f'len(Xtest6535_nosyn): {len(Xtest6535_nosyn)}')
print(f'len(Xtest6535_trig):  {len(Xtest6535_trig)}')
print(f'len(Xtest6535_ntrig): {len(Xtest6535_ntrig)}')

### Binary Classification: Balacing the Dataset

Generate the datasets for binary classification. 

In [ ]:
df = get_dataset(triggered, not_triggered)
df.loc[df['target'] != 0, 'target'] = 1

Xtrain8020_bin, Ytrain8020_bin, Xtest8020_bin, Ytest8020_bin = split_dataset(df, train_perc=0.8)
Xtrain6535_bin, Ytrain6535_bin, Xtest6535_bin, Ytest6535_bin = split_dataset(df, train_perc=0.65)

Ytrain8020_bin = preprocess_labels(Ytrain8020_bin, binary_classification=True)
Ytest8020_bin = preprocess_labels(Ytest8020_bin, binary_classification=True)
Ytrain6535_bin = preprocess_labels(Ytrain6535_bin, binary_classification=True)
Ytest6535_bin = preprocess_labels(Ytest6535_bin, binary_classification=True)

Ytest8020_bin

In [ ]:
df = get_dataset(triggered, not_triggered).drop(synthesis_features, axis=1)
df.loc[df['target'] != 0, 'target'] = 1

Xtrain8020_bin_nosyn, Ytrain8020_bin_nosyn, Xtest8020_bin_nosyn, Ytest8020_bin_nosyn = split_dataset(df, train_perc=0.8)
Xtrain6535_bin_nosyn, Ytrain6535_bin_nosyn, Xtest6535_bin_nosyn, Ytest6535_bin_nosyn = split_dataset(df, train_perc=0.65)

Ytrain8020_bin_nosyn = preprocess_labels(Ytrain8020_bin_nosyn, binary_classification=True)
Ytest8020_bin_nosyn = preprocess_labels(Ytest8020_bin_nosyn, binary_classification=True)
Ytrain6535_bin_nosyn = preprocess_labels(Ytrain6535_bin_nosyn, binary_classification=True)
Ytest6535_bin_nosyn = preprocess_labels(Ytest6535_bin_nosyn, binary_classification=True)

Ytest8020_bin_nosyn

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter

In [ ]:
ros = RandomOverSampler(random_state=0)

Xtrain8020_bin_resampled, Ytrain8020_bin_resampled = ros.fit_resample(Xtrain8020_bin, Ytrain8020_bin)
Xtrain6535_bin_resampled, Ytrain6535_bin_resampled = ros.fit_resample(Xtrain6535_bin, Ytrain6535_bin)

Xtrain8020_bin_resampled_nosyn, Ytrain8020_bin_resampled_nosyn = ros.fit_resample(Xtrain8020_bin_nosyn, Ytrain8020_bin_nosyn)
Xtrain6535_bin_resampled_nosyn, Ytrain6535_bin_resampled_nosyn = ros.fit_resample(Xtrain6535_bin_nosyn, Ytrain6535_bin_nosyn)

print(sorted(Counter(Ytrain6535_bin_resampled_nosyn).items()))

In [ ]:
dvec = DictVectorizer(sparse=False)
# SMOTE
X = dvec.fit_transform(Xtrain8020_bin.to_dict('records'))
Xtrain8020_bin_smote, Ytrain8020_bin_smote = SMOTE().fit_resample(X, Ytrain8020_bin)

X = dvec.fit_transform(Xtrain6535_bin.to_dict('records'))
Xtrain6535_bin_smote, Ytrain6535_bin_smote = SMOTE().fit_resample(X, Ytrain6535_bin)
print(sorted(Counter(Ytrain6535_bin_smote).items()))

# No synthesis features
X = dvec.fit_transform(Xtrain8020_bin_nosyn.to_dict('records'))
Xtrain8020_bin_smote_nosyn, Ytrain8020_bin_smote_nosyn = SMOTE().fit_resample(X, Ytrain8020_bin_nosyn)

X = dvec.fit_transform(Xtrain6535_bin_nosyn.to_dict('records'))
Xtrain6535_bin_smote_nosyn, Ytrain6535_bin_smote_nosyn = SMOTE().fit_resample(X, Ytrain6535_bin_nosyn)
print(sorted(Counter(Ytrain6535_bin_smote_nosyn).items()))

# ADASYN
X = dvec.fit_transform(Xtrain8020_bin.to_dict('records'))
Xtrain8020_bin_adasyn, Ytrain8020_bin_adasyn = ADASYN().fit_resample(X, Ytrain8020_bin)

X = dvec.fit_transform(Xtrain6535_bin.to_dict('records'))
Xtrain6535_bin_adasyn, Ytrain6535_bin_adasyn = ADASYN().fit_resample(X, Ytrain6535_bin)
print(sorted(Counter(Ytrain6535_bin_adasyn).items()))

# No synthesis features
X = dvec.fit_transform(Xtrain8020_bin_nosyn.to_dict('records'))
Xtrain8020_bin_adasyn_nosyn, Ytrain8020_bin_adasyn_nosyn = ADASYN().fit_resample(X, Ytrain8020_bin_nosyn)

X = dvec.fit_transform(Xtrain6535_bin_nosyn.to_dict('records'))
Xtrain6535_bin_adasyn_nosyn, Ytrain6535_bin_adasyn_nosyn = ADASYN().fit_resample(X, Ytrain6535_bin_nosyn)
print(sorted(Counter(Ytrain6535_bin_adasyn_nosyn).items()))

In [ ]:
# Vectorize Test datasets as well
dvec = DictVectorizer(sparse=False)

Xtest8020_bin_smote = dvec.fit_transform(Xtest8020_bin.to_dict('records'))
Xtest8020_bin_adasyn = dvec.fit_transform(Xtest8020_bin.to_dict('records'))
Xtest8020_bin_smote_nosyn = dvec.fit_transform(Xtest8020_bin_nosyn.to_dict('records'))
Xtest8020_bin_adasyn_nosyn = dvec.fit_transform(Xtest8020_bin_nosyn.to_dict('records'))

Xtest6535_bin_smote = dvec.fit_transform(Xtest6535_bin.to_dict('records'))
Xtest6535_bin_adasyn = dvec.fit_transform(Xtest6535_bin.to_dict('records'))
Xtest6535_bin_smote_nosyn = dvec.fit_transform(Xtest6535_bin_nosyn.to_dict('records'))
Xtest6535_bin_adasyn_nosyn = dvec.fit_transform(Xtest6535_bin_nosyn.to_dict('records'))

## Import SKlearn Modules

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.pipeline import Pipeline

from sklearn.model_selection import LeaveOneOut

# Features preprocessing
from sklearn.preprocessing import MinMaxScaler # Unused...
from sklearn.preprocessing import StandardScaler
# Dummy
from sklearn.dummy import DummyClassifier
# Linear classifiers
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC, SVC
# Bernoulli Model (requires binary vectorizer)
from sklearn.naive_bayes import BernoulliNB
# Ensemble(+Tree) Classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Neural network classifier (will take longer time to train)
from sklearn.neural_network import MLPClassifier

from sklearn.pipeline import Pipeline

## Machine Learning Models

In [ ]:
scoring_metrics = ['accuracy'] #, 'roc_auc']
models = {}
datasets = {
    # NOTE: All datasets include both triggered and non-triggered trojans
    # ==========================================================================
    # 80%-20% Training percentage
    # ==========================================================================
    '_8020': {
        'Xtrain': Xtrain8020,
        'Ytrain': Ytrain8020,
        'Xtest': Xtest8020,
        'Ytest': Ytest8020
    },
    '_8020_nosyn': {
        'Xtrain': Xtrain8020_nosyn,
        'Ytrain': Ytrain8020_nosyn,
        'Xtest': Xtest8020_nosyn,
        'Ytest': Ytest8020_nosyn
    },
    # '_8020_trig': {
    #     'Xtrain': Xtrain8020_trig,
    #     'Ytrain': Ytrain8020_trig,
    #     'Xtest': Xtest8020_trig,
    #     'Ytest': Ytest8020_trig
    # },
    # '_8020_ntrig': {
    #     'Xtrain': Xtrain8020_ntrig,
    #     'Ytrain': Ytrain8020_ntrig,
    #     'Xtest': Xtest8020_ntrig,
    #     'Ytest': Ytest8020_ntrig
    # },
    '_8020_bin_resampled': {
        'Xtrain': Xtrain8020_bin_resampled,
        'Xtest': Xtest8020_bin,
        'Ytrain': Ytrain8020_bin_resampled,
        'Ytest': Ytest8020_bin
    },
    # '_8020_bin_smote': {
    #     'Xtrain': Xtrain8020_bin_smote,
    #     'Xtest': Xtest8020_bin_smote,
    #     'Ytrain': Ytrain8020_bin_smote,
    #     'Ytest': Ytest8020_bin
    # },
    # '_8020_bin_adasyn': {
    #     'Xtrain': Xtrain8020_bin_adasyn,
    #     'Xtest': Xtest8020_bin_adasyn,
    #     'Ytrain': Ytrain8020_bin_adasyn,
    #     'Ytest': Ytest8020_bin
    # },
    '_8020_bin_resampled_nosyn': {
        'Xtrain': Xtrain8020_bin_resampled_nosyn,
        'Xtest': Xtest8020_bin_nosyn,
        'Ytrain': Ytrain8020_bin_resampled_nosyn,
        'Ytest': Ytest8020_bin_nosyn
    },
    # '_8020_bin_smote_nosyn': {
    #     'Xtrain': Xtrain8020_bin_smote_nosyn,
    #     'Xtest': Xtest8020_bin_smote_nosyn,
    #     'Ytrain': Ytrain8020_bin_smote_nosyn,
    #     'Ytest': Ytest8020_bin_nosyn
    # },
    # '_8020_bin_adasyn_nosyn': {
    #     'Xtrain': Xtrain8020_bin_adasyn_nosyn,
    #     'Xtest': Xtest8020_bin_adasyn_nosyn,
    #     'Ytrain': Ytrain8020_bin_adasyn_nosyn,
    #     'Ytest': Ytest8020_bin_nosyn
    # },
    # ==========================================================================
    # 65%-35% Training percentage
    # ==========================================================================
    '_6535': {
        'Xtrain': Xtrain6535,
        'Ytrain': Ytrain6535,
        'Xtest': Xtest6535,
        'Ytest': Ytest6535
    },
    '_6535_nosyn': {
        'Xtrain': Xtrain6535_nosyn,
        'Ytrain': Ytrain6535_nosyn,
        'Xtest': Xtest6535_nosyn,
        'Ytest': Ytest6535_nosyn
    },
    # '_6535_trig': {
    #     'Xtrain': Xtrain6535_trig,
    #     'Ytrain': Ytrain6535_trig,
    #     'Xtest': Xtest6535_trig,
    #     'Ytest': Ytest6535_trig
    # },
    # '_6535_ntrig': {
    #     'Xtrain': Xtrain6535_ntrig,
    #     'Ytrain': Ytrain6535_ntrig,
    #     'Xtest': Xtest6535_ntrig,
    #     'Ytest': Ytest6535_ntrig
    # },
    '_6535_bin_resampled': {
        'Xtrain': Xtrain6535_bin_resampled,
        'Xtest': Xtest6535_bin,
        'Ytrain': Ytrain6535_bin_resampled,
        'Ytest': Ytest6535_bin
    },
    # '_6535_bin_smote': {
    #     'Xtrain': Xtrain6535_bin_smote,
    #     'Xtest': Xtest6535_bin_smote,
    #     'Ytrain': Ytrain6535_bin_smote,
    #     'Ytest': Ytest6535_bin
    # },
    # '_6535_bin_adasyn': {
    #     'Xtrain': Xtrain6535_bin_adasyn,
    #     'Xtest': Xtest6535_bin_adasyn,
    #     'Ytrain': Ytrain6535_bin_adasyn,
    #     'Ytest': Ytest6535_bin
    # },
    '_6535_bin_resampled_nosyn': {
        'Xtrain': Xtrain6535_bin_resampled_nosyn,
        'Xtest': Xtest6535_bin_nosyn,
        'Ytrain': Ytrain6535_bin_resampled_nosyn,
        'Ytest': Ytest6535_bin_nosyn
    },
    # '_6535_bin_smote_nosyn': {
    #     'Xtrain': Xtrain6535_bin_smote_nosyn,
    #     'Xtest': Xtest6535_bin_smote_nosyn,
    #     'Ytrain': Ytrain6535_bin_smote_nosyn,
    #     'Ytest': Ytest6535_bin_nosyn
    # },
    # '_6535_bin_adasyn_nosyn': {
    #     'Xtrain': Xtrain6535_bin_adasyn_nosyn,
    #     'Xtest': Xtest6535_bin_adasyn_nosyn,
    #     'Ytrain': Ytrain6535_bin_adasyn_nosyn,
    #     'Ytest': Ytest6535_bin_nosyn
    # },
}

### Dummy Classifier (Baseline)

In [ ]:
def make_model():
    # Setup the pipeline with the vectorizer and the classifier
    pipeline = Pipeline([
        ('vect', DictVectorizer(sparse=False)),
        ('clf', DummyClassifier()),
    ])
    # Add the classifier configurations to the grid of parameters
    parameters = {
        'clf__strategy': ('most_frequent', 'prior'),
    }
    return GridSearchCV(pipeline, parameters,
                        scoring=scoring_metrics,
                        refit='accuracy',
                        cv=5,
                        error_score='raise',
                        n_jobs=-1,
                        verbose=3)

for dataset_type in datasets.keys():
    # Add the grid-search to the list of models
    model_name = 'Dummy'
    models[f'{model_name}{dataset_type}'] = {}
    # NOTE: For the resampled datasets, the train data are already vectorized 
    if '_smote' in dataset_type or '_adasyn' in dataset_type:
        models[f'{model_name}{dataset_type}']['model'] = make_model()
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain']
        models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest']
    else:
        models[f'{model_name}{dataset_type}']['model'] = make_model()
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain'].to_dict('records')
        models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytrain'] = datasets[dataset_type]['Ytrain']
    models[f'{model_name}{dataset_type}']['Ytest'] = datasets[dataset_type]['Ytest']

### Decision Tree Classifier

In [ ]:
def make_model():
    # Setup the pipeline with the vectorizer and the classifier
    pipeline = Pipeline([
        ('vect', DictVectorizer(sparse=False)),
        ('clf', DecisionTreeClassifier()),
    ])
    # Add the classifier configurations to the grid of parameters
    parameters = {
        'clf__max_depth': randint(32, 256),
    }
    return  RandomizedSearchCV(pipeline,
                               parameters,
                               scoring=scoring_metrics,
                               refit='accuracy',
                               cv=5,
                               error_score='raise',
                               n_jobs=-1,
                               random_state=0,
                               verbose=3)

for dataset_type in datasets.keys():
    # Add the grid-search to the list of models
    model_name = 'DecisionTree'
    models[f'{model_name}{dataset_type}'] = {}
    models[f'{model_name}{dataset_type}']['model'] = make_model()
    if '_smote' in dataset_type or '_adasyn' in dataset_type:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain']
    else:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytrain'] = datasets[dataset_type]['Ytrain']
    models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytest'] = datasets[dataset_type]['Ytest']

### Random Forest Classifier

For some unkown reason, the cross-validation of random forests with `RandomizedSearchCV` takes a long time to finish and returns zero scores. Because of that, I'm wrapping a _fixed_ pipeline with a class handling the required methods.

In [ ]:
from sklearn.model_selection import cross_validate
import numpy as np

class RFClassifier(object):
    def __init__(self):
        parameters = {
            'clf__n_estimators': randint(100, 128),
            'clf__max_depth': randint(64, 256),
        }
        self.best_estimator_ = Pipeline([
                                        ('vect', DictVectorizer(sparse=False)),
                                        ('clf', RandomForestClassifier(n_estimators=256, max_depth=128)),
                                    ])
        # self.best_estimator_ = RandomizedSearchCV(pipeline,
        #                                           parameters,
        #                                           scoring=scoring_metrics,
        #                                           refit='accuracy',
        #                                           cv=5,
        #                                           n_iter=10,
        #                                           error_score='raise',
        #                                           n_jobs=-1,
        #                                           random_state=0,
        #                                           verbose=3)
        self.best_score_ = None
        self.classes_ = hth_classes

    def fit(self, X, Y):
        self.cv_results_ = cross_validate(self.best_estimator_, X, y=Y,
                                              scoring='accuracy',
                                              cv=5,
                                              n_jobs=-1)
        self.best_score_ = np.mean(self.cv_results_['test_score'])
        self.best_estimator_.fit(X, Y)

    def score(self, X, Y):
        return self.best_estimator_.score(X, Y)
    
    def predict(self, X):
        return self.best_estimator_.predict(X)

for dataset_type in datasets.keys():
    # Add the grid-search to the list of models
    model_name = 'RandomForest'
    models[f'{model_name}{dataset_type}'] = {}
    models[f'{model_name}{dataset_type}']['model'] = make_model()
    # NOTE: For the resampled datasets, the train data are already vectorized 
    if '_smote' in dataset_type or '_adasyn' in dataset_type:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain']
    else:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytrain'] = datasets[dataset_type]['Ytrain']
    models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytest'] = datasets[dataset_type]['Ytest']

### Logistic Regression

In [ ]:
def make_model():
    # Setup the pipeline with the vectorizer and the classifier
    pipeline = Pipeline([
        ('vect', DictVectorizer(sparse=False)),
        ('clf', LogisticRegression(max_iter=500)),
    ])
    # Add the classifier configurations to the grid of parameters
    parameters = {
        'clf__C': uniform(0.6, 0.95),
        'clf__solver': ('newton-cg', 'lbfgs', 'liblinear'),
    }
    # Add the grid-search to the list of models
    return RandomizedSearchCV(pipeline,
                              parameters,
                              scoring=scoring_metrics,
                              refit='accuracy',
                              cv=5,
                              error_score='raise',
                              n_jobs=-1,
                              random_state=0,
                              verbose=3)

for dataset_type in datasets.keys():
    # Add the grid-search to the list of models
    model_name = 'LogisticRegression'
    models[f'{model_name}{dataset_type}'] = {}
    models[f'{model_name}{dataset_type}']['model'] = make_model()
    # NOTE: For the resampled datasets, the train data are already vectorized 
    if '_smote' in dataset_type or '_adasyn' in dataset_type:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain']
    else:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytrain'] = datasets[dataset_type]['Ytrain']
    models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytest'] = datasets[dataset_type]['Ytest']

### Linear SVC

In [ ]:
def make_model():
    # Setup the pipeline with the vectorizer and the classifier
    pipeline = Pipeline([
        ('vect', DictVectorizer(sparse=False)),
        ('clf', LinearSVC(dual=True, max_iter=2000)),
    ])
    # Add the classifier configurations to the grid of parameters
    parameters = {
        'clf__loss': ('hinge', 'squared_hinge'),
        'clf__C': uniform(0.4, 0.95),
    }
    # Add the grid-search to the list of models
    return RandomizedSearchCV(pipeline,
                              parameters,
                              scoring=scoring_metrics,
                              refit='accuracy',
                              cv=5,
                              error_score='raise',
                              n_jobs=-1,
                              random_state=0,
                              verbose=3)

for dataset_type in datasets.keys():
    # Add the grid-search to the list of models
    model_name = 'LinearSVC'
    models[f'{model_name}{dataset_type}'] = {}
    models[f'{model_name}{dataset_type}']['model'] = make_model()
    # NOTE: For the resampled datasets, the train data are already vectorized 
    if '_smote' in dataset_type or '_adasyn' in dataset_type:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain']
    else:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytrain'] = datasets[dataset_type]['Ytrain']
    models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytest'] = datasets[dataset_type]['Ytest']

###  Gradient Boosting

In [ ]:
def make_model():
    # Setup the pipeline with the vectorizer and the classifier
    pipeline = Pipeline([
        ('vect', DictVectorizer(sparse=False)),
        ('clf', GradientBoostingClassifier(max_depth=8)),
    ])
    # Add the classifier configurations to the grid of parameters
    parameters = {
        'clf__n_estimators': randint(80, 120),
        'clf__learning_rate': uniform(0.01, 0.1),
    }
    # Add the grid-search to the list of models
    return RandomizedSearchCV(pipeline,
                              parameters,
                              scoring=scoring_metrics,
                              refit='accuracy',
                              cv=5,
                              error_score='raise',
                              n_jobs=-1,
                              random_state=0,
                              verbose=3)

for dataset_type in datasets.keys():
    # Add the grid-search to the list of models
    model_name = 'GradientBoosting'
    models[f'{model_name}{dataset_type}'] = {}
    models[f'{model_name}{dataset_type}']['model'] = make_model()
    # NOTE: For the resampled datasets, the train data are already vectorized 
    if '_smote' in dataset_type or '_adasyn' in dataset_type:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain']
    else:
        models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytrain'] = datasets[dataset_type]['Ytrain']
    models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest'].to_dict('records')
    models[f'{model_name}{dataset_type}']['Ytest'] = datasets[dataset_type]['Ytest']

## Deep Learning Models


### MLP Classifier with SKlearn

In [ ]:
# from scipy.stats import randint as sp_randint

# def make_model():
#     # Setup the pipeline with the vectorizer and the classifier
#     pipeline = Pipeline([
#         ('vect', DictVectorizer(sparse=False)),
#         ('norm', StandardScaler()),
#         ('clf', MLPClassifier(learning_rate_init=0.01, max_iter=500)),
#     ])
#     # Add the classifier configurations to the grid of parameters

#     # TODO: How do I randomize the layer number search???
#     parameters = {
#         'clf__hidden_layer_sizes': [(128, 128),(128, 128, 128),],
#     }
#     # Add the grid-search to the list of models
#     return GridSearchCV(pipeline,
#                         parameters,
#                         scoring=scoring_metrics,
#                         refit='accuracy',
#                         cv=5,
#                         error_score='raise',
#                         n_jobs=-1,
#                         verbose=3)

# for dataset_type in datasets.keys():
#     # Add the grid-search to the list of models
#     model_name = 'MLP'
#     models[f'{model_name}{dataset_type}'] = {}
#     models[f'{model_name}{dataset_type}']['model'] = make_model()
#     # NOTE: For the resampled datasets, the train data are already vectorized 
#     if '_smote' in dataset_type or '_adasyn' in dataset_type:
#         models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain']
#     else:
#         models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain'].to_dict('records')
#     models[f'{model_name}{dataset_type}']['Ytrain'] = datasets[dataset_type]['Ytrain']
#     models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest'].to_dict('records')
#     models[f'{model_name}{dataset_type}']['Ytest'] = datasets[dataset_type]['Ytest']

### MLP Classifier with Keras

In order to fully exploit Sklearn developing flow, we utilize Keras model wrapper provided by `scikeras`.

The idea is to define a custom Keras model and integrate it into a Sklearn pipeline. In this way, we don't need to worry about fine-grain details like input/output shape matching and we can exploit the CV search.

More information and tutorials can be found [here](https://www.adriangb.com/scikeras/stable/notebooks/MLPClassifier_MLPRegressor.html).

In [ ]:
# from typing import Dict, Iterable, Any

# def get_clf_model(num_hidden_layers: int,
#                   hidden_layer_sizes: int,
#                   meta: Dict[str, Any],
#                   compile_kwargs: Dict[str, Any]):
#     # Generate a model and populate it according to the passed arguments
#     model = keras.Sequential()
#     inp = keras.layers.Input(shape=(meta['n_features_in_']))
#     model.add(inp)
#     for _ in range(num_hidden_layers):
#         l2 = tf.keras.regularizers.l2(0.001)
#         layer = Dense(hidden_layer_sizes, activation='relu', kernel_regularizer=l2)
#         model.add(layer)
#     # The final layer and the loss function will depend on the kind of task we
#     # are aiming for (Sklearn will automatically determine it and pass the
#     # information via the `meta` dictionary).
#     if meta['target_type_'] == 'binary':
#         n_output_units = 1
#         output_activation = 'sigmoid'
#         loss = 'binary_crossentropy'
#     elif meta['target_type_'] == 'multiclass':
#         n_output_units = meta['n_classes_']
#         output_activation = 'softmax'
#         loss = 'sparse_categorical_crossentropy'
#     else:
#         raise NotImplementedError(f'Unsupported task type: {meta["target_type_"]}')
#     # Add the final layer and compile the model
#     out = Dense(n_output_units, activation=output_activation)
#     model.add(out)
#     model.compile(loss=loss, optimizer=compile_kwargs['optimizer'])
#     return model

# def make_model():
#     # Setup the pipeline with the vectorizer and the classifier
#     # NOTE: We are using a Sklearn wrapper for the Keras model
#     pipeline = Pipeline([
#         ('vect', DictVectorizer(sparse=False)),
#         ('norm', StandardScaler()),
#         ('clf', KerasClassifier(model=get_clf_model,
#                                 optimizer='adam',
#                                 optimizer__learning_rate=0.1,
#                                 epochs=50,
#                                 batch_size=32)),
#     ])
#     # NOTE: Passing specific parameters to the model generation function should
#     # be done by specifying the "model__" keyword.
#     parameters = {
#         'clf__model__hidden_layer_sizes': randint(128, 256),
#         'clf__model__num_hidden_layers': randint(2, 3),
#     }
#     return RandomizedSearchCV(pipeline,
#                               parameters,
#                               scoring=scoring_metrics,
#                               refit='accuracy',
#                               cv=5,
#                               error_score='raise',
#                               n_jobs=-1,
#                               random_state=0,
#                               verbose=3)

# for dataset_type in datasets.keys():
#     # Add the grid-search to the list of models
#     model_name = 'MLPKeras'
#     models[f'{model_name}{dataset_type}'] = {}
#     models[f'{model_name}{dataset_type}']['model'] = make_model()
#     # NOTE: For the resampled datasets, the train data are already vectorized 
#     if '_smote' in dataset_type or '_adasyn' in dataset_type:
#         models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain']
#     else:
#         models[f'{model_name}{dataset_type}']['Xtrain'] = datasets[dataset_type]['Xtrain'].to_dict('records')
#     models[f'{model_name}{dataset_type}']['Ytrain'] = datasets[dataset_type]['Ytrain']
#     models[f'{model_name}{dataset_type}']['Xtest'] = datasets[dataset_type]['Xtest'].to_dict('records')
#     models[f'{model_name}{dataset_type}']['Ytest'] = datasets[dataset_type]['Ytest']

In [ ]:
# from __future__ import print_function
# import tensorflow as tf
# import keras
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
# from keras import Input

# from sklearn.model_selection import RepeatedKFold, cross_val_score
# from scikeras.wrappers import KerasClassifier

In [ ]:
# %%capture
# try:
#     import scikeras
# except ImportError:
#     !python -m pip install scikeras

## Cross-Validation and Training

In [ ]:
from sklearn.utils.validation import check_is_fitted
from joblib import dump, load

SCORE_THRESHOLD = 0.0
LOAD_FROM_DISK = True

scores = {}
for model_type in models.keys():
    modelfile = os.path.join(models_dir, f'{model_type}.joblib')
    if LOAD_FROM_DISK:
        # Check if model exists on disk and eventually load it.
        if os.path.exists(modelfile):
            models[model_type]['model'] = load(modelfile)
            print(f'Successfully loaded model: {model_type}')
            model_saved = True
        else:
            model_saved = False
    else:
        model_saved = False
    # Check if model is fitted and if so, that its score is above a threshold.
    try:
        model = models[model_type]['model']
        x_test = models[model_type]['Xtest']
        y_test = models[model_type]['Ytest']
        check_is_fitted(model)
        model_trained = model.score(x_test, y_test) > SCORE_THRESHOLD
        print(f'Model {model_type} not achieving high score, attempting retraining.')
    except:
        print(f'Model {model_type} not fitted.')
        model_trained = False
    # Fit if the performance of the model are low or if it has not been loaded
    # from disk.
    if not model_trained or not model_saved:
        print(f'INFO. Fitting model: {model_type}')
        model = models[model_type]['model']
        x_train = models[model_type]['Xtrain']
        y_train = models[model_type]['Ytrain']
        model.fit(x_train, y_train)
        dump(model, modelfile)
    scores[model_type] = score = models[model_type]['model'].best_score_
    print(f'INFO. {model_type} model reached CV score of: {score:.3f}')
    print('-' * 80)

In [ ]:
table_data = []
for model_type in models.keys():
    table_data.append((model_type, models[model_type]['model'].best_score_))
pd.DataFrame(table_data, columns=['Model', 'CV Score'])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

def clean_name(s):
    return s.replace('_bin_resampled', '') \
            .replace('_8020', ' 80%-20%') \
            .replace('_6535', ' 65%-35%') \
            .replace('_nosyn', '')

full_scores = [scores[k] for k in models.keys() if '_nosyn' not in k and '_bin' not in k]
nosyn_scores = [scores[k] for k in models.keys() if '_nosyn' in k and '_bin' not in k]
keys = [clean_name(k) for k in models.keys() if '_nosyn' not in k and '_bin' not in k]

x = np.array([x for x in range(len(keys))])
plt.bar(x, full_scores, color=['C1' if s > 0.8 else 'C0' for s in full_scores])
# plt.bar(-x, full_scores, color='C0', label='PF + SF')
# plt.bar(x, nosyn_scores, color='C1', label='SF')

plt.xticks(x, [f'{m}' for m in keys], rotation=90)

# ax.set_yticks([0.25, 0.75, 0.9], minor=True)
# ax.set_yticklabels([0.25, 0.75, 0.9], minor=True)
plt.grid(which='both', axis='y', alpha=0.7, zorder=1)

plt.ylabel('Accuracy')
# plt.title('CV Accuracy score of the selected classifiers')
plt.tight_layout()
# plt.legend()
plt.savefig(os.path.join(fig_dir, f'cv_accuracy.pdf'))
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

# score_val = scores.values()
# keys = models.keys()
score_val = [scores[k] for k in models.keys() if '_nosyn' not in k and '_bin' in k]
keys = [clean_name(k) for k in models.keys() if '_nosyn' not in k and '_bin' not in k]


linspace = [x for x in range(len(keys))]
plt.bar(linspace, score_val, color=['C1' if s > 0.8 else 'C0' for s in score_val])
plt.xticks(linspace, [f'{m}' for m in keys], rotation=90)

# ax.set_yticks([0.25, 0.75, 0.9], minor=True)
# ax.set_yticklabels([0.25, 0.75, 0.9], minor=True)
plt.grid(which='both', axis='y', alpha=0.7, zorder=1)

plt.ylabel('Accuracy')
plt.title('CV Accuracy score of the selected binary classifiers')
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, f'cv_accuracy_bin.pdf'))
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

# score_val = scores.values()
# keys = models.keys()
score_val = [scores[k] for k in models.keys() if '_nosyn' in k and '_bin' not in k]
keys = [clean_name(k) for k in models.keys() if '_nosyn' in k and '_bin' not in k]

linspace = [x for x in range(len(keys))]
plt.bar(linspace, score_val, color=['C1' if s > 0.8 else 'C0' for s in score_val])
plt.xticks(linspace, [f'{m}' for m in keys], rotation=90)

# ax.set_yticks([0.25, 0.75, 0.9], minor=True)
# ax.set_yticklabels([0.25, 0.75, 0.9], minor=True)
plt.grid(which='both', axis='y', alpha=0.7, zorder=1)
plt.ylim([0, 1])

plt.ylabel('Accuracy')
plt.title('CV Accuracy score of the selected classifiers')
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, f'cv_accuracy_nosyn.pdf'))
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

score_val = [scores[k] for k in models.keys() if '_nosyn' in k and '_bin' in k]
keys = [clean_name(k) for k in models.keys() if '_nosyn' in k and '_bin' in k]

linspace = [x for x in range(len(keys))]
plt.bar(linspace, score_val, color=['C1' if s > 0.8 else 'C0' for s in score_val])
plt.xticks(linspace, [f'{m}' for m in keys], rotation=90)

# ax.set_yticks([0.25, 0.75, 0.9], minor=True)
# ax.set_yticklabels([0.25, 0.75, 0.9], minor=True)
plt.grid(which='both', axis='y', alpha=0.7, zorder=1)
plt.ylim([0, 1])

plt.ylabel('Accuracy')
plt.title('CV Accuracy score of the selected binary classifiers')
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, f'cv_accuracy_nosyn_bin.pdf'))
plt.show()

### Inspect Best Models from Random Grid Search

In [ ]:
# if 'MLPKeras_base' in models:
#     models['MLPKeras_base']['model'].best_estimator_['clf'].model_.summary()

In [ ]:
# if 'RandomForest_base' in models:
#     models['RandomForest_base']['model'].best_estimator_['clf'].classes_

In [ ]:
# if 'RandomForest_base' in models:
#     models['RandomForest_base']['model'].cv_results_

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

if 'RandomForest_base' in models:
    tree = models['RandomForest_base']['model'].best_estimator_['clf'].estimators_[0]
    plot_tree(tree, 
            #   feature_names=dv.feature_names_,
            class_names=hth_classes,
            filled=True, rounded=True)
    plt.savefig(os.path.join(fig_dir, f'tree.pdf'))
    plt.show()

In [ ]:
# import numpy as np

# def get_roc_auc(model_type):
#     i = np.argmin(models[model_type].cv_results_['rank_test_roc_auc'])
#     return models[model_type].cv_results_['mean_test_roc_auc'][i]

# roc_auc_scores = {}
# for model_type in models.keys():
#     roc_auc_scores[model_type] = get_roc_auc(model_type)
#     print(f'{model_type} ROC-AUC CV score: {get_roc_auc(model_type):.3f}')

# linspace = [x for x in range(len(models.keys()))]
# plt.bar(linspace, roc_auc_scores.values())
# plt.xticks(linspace, [f'{m}' for m in models.keys()], rotation=90)
# plt.grid(which='both', axis='y', alpha=0.7, zorder=1)
# plt.ylabel('Accuracy')
# plt.title('ROC-AUC CV Accuracy score of different classifiers')
# plt.savefig(os.path.join(data_dir, f'cv_roc_auc.pdf'))
# plt.tight_layout()
# plt.show()

## Evaluation

First collect all the scores, then plot and display them accordingly.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

ROUNDING = 2
train_test_split = {
    '8020': '80%-20%',
    '6535': '65%-35%',
}
table_data = []
accuracy_scores = {}
for model_type in models.keys():
    model = models[model_type]['model']
    x_test = models[model_type]['Xtest']
    y_test = models[model_type]['Ytest']
    cv_accuracy = round(model.best_score_, ROUNDING)
    y_pred = model.predict(x_test)
    accuracy = round(model.score(x_test, y_test), ROUNDING)
    accuracy_str = f'{accuracy:.4f}'
    m_str = model_type
    if '_bin' in model_type:
        binary_classification = True
    else:
        binary_classification = False
    if '_nosyn' in model_type:
        # continue
        included_featues = 'IF'
        m_str = m_str.replace('_nosyn', '')
    else:
        included_featues = 'PF + IF'
    # Get information about the model type and extract reporting model ID label
    if '_ntrig' in model_type:
        triggered_dataset = 'False'
        m_str = m_str.replace('_ntrig', '')
        train_perc = m_str.split('_')[1]
        model_str = m_str.split('_')[0]
        dummy_type = '_' + train_perc if triggered_dataset != 'Both' else ''
        dummy_type += '_ntrig'
    elif '_trig' in model_type:
        triggered_dataset = 'True'
        m_str = m_str.replace('_trig', '')
        train_perc = m_str.split('_')[1]
        model_str = m_str.split('_')[0]
        dummy_type = '_' + train_perc if triggered_dataset != 'Both' else ''
        dummy_type += '_trig'
    else:
        triggered_dataset = 'Both'
        train_perc = m_str.split('_')[1]
        model_str = m_str.split('_')[0]
        dummy_type = '_' + train_perc
    dummy_type += '_nosyn' if '_nosyn' in model_type else ''
    # Compute additional scores
    other_scores = precision_recall_fscore_support(y_test, y_pred,
                                                    average='micro',
                                                    # labels=hth_classes,
                                                    zero_division=0)
    # Do not consider the support, i.e. the last score
    other_scores = [round(s, ROUNDING) if s is not None else 0 for s in other_scores]
    # Calculate Cohen's Kappa
    if 'Dummy' not in model_type:
        dummy_score = models[f'Dummy{dummy_type}']['model'].score(x_test, y_test)
        cohens_kappa = (accuracy - dummy_score) / (1.0 - dummy_score)
        cohens_kappa = round(cohens_kappa, ROUNDING)
    else:
        cohens_kappa = 0
    model_str_split = f'{model_str} ({train_test_split[train_perc]})'
    table_entry = (model_str, model_str_split, train_test_split[train_perc],
                   included_featues, triggered_dataset, binary_classification,
                   cv_accuracy, accuracy, *other_scores, cohens_kappa)
    table_data.append(table_entry)
    accuracy_scores[model_type] = accuracy

cols = ['Model', 'Model and Split', 'Train/Test Split', 'Included Features',
        'Include Triggered', 'Binary C.', 'CV Accuracy', 'Accuracy',
        'Precision', 'Recall', 'Fbeta', 'Support', 'k']
table = pd.DataFrame(table_data, columns=cols).drop(['Support'], axis=1)
table.to_csv(os.path.join(data_dir, f'results.csv'), encoding='utf-8', index=False)

In [ ]:
table

### Score Plots

In [ ]:
import matplotlib

# data = table[table['Binary C.'] == False]

# fig, ax =plt.subplots(1, 2)
# sns.barplot(data=data, y='Model', x='Accuracy', hue='Included Features', orient='h', ax=ax[0])
# sns.barplot(data=data, y='Model', x='k', hue='Included Features', orient='h', ax=ax[1])
# sns.move_legend(ax[1], 'upper left', bbox_to_anchor=(1, 1))

# for container in ax[0].containers:
#     ax[0].bar_label(container, padding=10)

# for container in ax[1].containers:
#     ax[1].bar_label(container, padding=10)

# plt.grid(axis='x')
# plt.xlim([0, 1.1])
# fig.savefig(os.path.join(fig_dir, f'eval_scores.pdf'), bbox_inches='tight')
# fig.show()

In [ ]:
data = table[table['Binary C.'] == False]
ax = sns.barplot(data=data, y='Model', x='CV Accuracy', hue='Included Features', orient='h')
# sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 1))
sns.move_legend(ax, 'lower left', bbox_to_anchor=(0, 1, 1, 0))

for container in ax.containers:
    ax.bar_label(container, padding=10)

plt.grid(axis='x')
plt.xlim([0, 1.1])
plt.savefig(os.path.join(fig_dir, f'cv_accuracy.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
data = table[table['Binary C.'] == False]
ax = sns.barplot(data=data, y='Model and Split', x='Accuracy', hue='Included Features', orient='h')
# sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 1))
sns.move_legend(ax, 'lower left', bbox_to_anchor=(0, 1, 1, 0))

for container in ax.containers:
    ax.bar_label(container, padding=10)

plt.grid(axis='x')
plt.xlim([0, 1.1])
plt.savefig(os.path.join(fig_dir, f'eval_accuracy_split.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
data = table[table['Binary C.'] == False]
ax = sns.barplot(data=data, y='Model', x='Accuracy', hue='Included Features', orient='h')
# sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 1))
sns.move_legend(ax, 'lower left', bbox_to_anchor=(0, 1, 1, 0))

for container in ax.containers:
    ax.bar_label(container, padding=10)

plt.grid(axis='x')
plt.xlim([0, 1.15])
plt.savefig(os.path.join(fig_dir, f'eval_accuracy.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
data = table[table['Binary C.'] == False]
ax = sns.barplot(data=data, y='Model', x='k', hue='Included Features', orient='h')
# sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 1))
sns.move_legend(ax, 'lower left', bbox_to_anchor=(0, 1, 1, 0))

for container in ax.containers:
    ax.bar_label(container, padding=10)

plt.grid(axis='x')
plt.xlim([0, 1.1])
plt.savefig(os.path.join(fig_dir, f'eval_k.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
data = table[table['Binary C.'] == True]
ax = sns.barplot(data=data, y='Model', x='Fbeta', hue='Included Features', orient='h')
# sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 1))
sns.move_legend(ax, 'lower left', bbox_to_anchor=(0, 1, 1, 0))

for container in ax.containers:
    ax.bar_label(container, padding=10)

ax.grid(axis='x')
plt.xlim([0, 1.1])

fig = ax.get_figure()
plt.savefig(os.path.join(fig_dir, f'eval_accuracy_bin_Fbeta.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
data = table[table['Binary C.'] == True]
ax = sns.barplot(data=data, y='Model', x='Accuracy', hue='Included Features', orient='h')
# sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 1))
sns.move_legend(ax, 'lower left', bbox_to_anchor=(0, 1, 1, 0))

for container in ax.containers:
    ax.bar_label(container, padding=10)

ax.grid(axis='x')
plt.xlim([0, 1.1])

fig = ax.get_figure()
plt.savefig(os.path.join(fig_dir, f'eval_accuracy_bin.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
data = table[table['Binary C.'] == True]
ax = sns.barplot(data=data, y='Model and Split', x='Accuracy', hue='Included Features', orient='h')
# sns.move_legend(ax, 'upper left', bbox_to_anchor=(1, 1))
sns.move_legend(ax, 'lower left', bbox_to_anchor=(0, 1, 1, 0))

for container in ax.containers:
    ax.bar_label(container, padding=10)

ax.grid(axis='x')
plt.xlim([0, 1.1])

fig = ax.get_figure()
plt.savefig(os.path.join(fig_dir, f'eval_accuracy_bin_split.pdf'), bbox_inches='tight')
plt.show()

### ROC-AUC Plots

In [ ]:
import sklearn.metrics as metrics
from sklearn.calibration import CalibratedClassifierCV

roc_curves = []
# Calculate the FPR and TPR for all thresholds of the classification
for model_type in models.keys():
    if '_bin' not in model_type:
        continue
    model = models[model_type]['model']
    x_test = models[model_type]['Xtest']
    y_test = models[model_type]['Ytest']

    try:
        probs = model.predict_proba(x_test)
        preds = probs[:, 1]
        fpr, tpr, threshold = metrics.roc_curve(y_test, preds, pos_label='with_hth')
        roc_auc = metrics.auc(fpr, tpr)
        roc_curves.append((model_type, fpr, tpr, roc_auc))
    except Exception as e:
        print(f'* Model {model_type} raised: {e}')

        if 'LinearSVC' in model_type:
            clf = CalibratedClassifierCV(model)
            clf.fit(models[model_type]['Xtrain'], models[model_type]['Ytrain'])
            y_proba = clf.predict_proba(x_test)
            probs = clf.predict_proba(x_test)
            preds = probs[:, 1]
            fpr, tpr, threshold = metrics.roc_curve(y_test, preds, pos_label='with_hth')
            roc_auc = metrics.auc(fpr, tpr)
            roc_curves.append((model_type, fpr, tpr, roc_auc))
        continue

for model_type, fpr, tpr, roc_auc in roc_curves:
    if '_nosyn' in model_type and '_8020' in model_type:
        model_type = model_type.replace('_8020', '')
        plt.plot(fpr, tpr, label=f'{clean_name(model_type)} - AUC={roc_auc:0.2f}')
# plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
# plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.04, 1.04])
plt.ylim([-0.04, 1.04])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.grid('both')
plt.savefig(os.path.join(fig_dir, f'roc-auc_nosyn_8020.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
for model_type, fpr, tpr, roc_auc in roc_curves:
    if '_nosyn' in model_type and '_6535' in model_type:
        model_type = model_type.replace('_6535', '')
        plt.plot(fpr, tpr, label=f'{clean_name(model_type)} - AUC={roc_auc:0.2f}')
# plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
# plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.04, 1.04])
plt.ylim([-0.04, 1.04])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.grid('both')
plt.savefig(os.path.join(fig_dir, f'roc-auc_nosyn_6535.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
for model_type, fpr, tpr, roc_auc in roc_curves:
    if '_nosyn' not in model_type and '_8020' in model_type:
        model_type = model_type.replace('_8020', '')
        plt.plot(fpr, tpr, label=f'{clean_name(model_type)} - AUC={roc_auc:0.2f}')
# plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
# plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.04, 1.04])
plt.ylim([-0.04, 1.04])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.grid('both')
plt.savefig(os.path.join(fig_dir, f'roc-auc_8020.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
for model_type, fpr, tpr, roc_auc in roc_curves:
    if '_nosyn' not in model_type and '_6535' in model_type:
        model_type = model_type.replace('_6535', '')
        plt.plot(fpr, tpr, label=f'{clean_name(model_type)} - AUC={roc_auc:0.2f}')
# plt.title('Receiver Operating Characteristic')
plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.04, 1.04])
plt.ylim([-0.04, 1.04])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.grid('both')
plt.savefig(os.path.join(fig_dir, f'roc-auc_6535.pdf'), bbox_inches='tight')
plt.show()

### Accuracy Plots (Deprecated)

In [ ]:
fig, ax = plt.subplots()

# score_val = accuracy_scores.values()
# keys = models.keys()
score_val = [accuracy_scores[k] for k in models.keys() if '_nosyn' not in k and '_bin' not in k]
keys = [k for k in models.keys() if '_nosyn' not in k and '_bin' not in k]

linspace = [x for x in range(len(keys))]
plt.bar(linspace, score_val, color=['C1' if s > 0.75 else 'C0' for s in score_val])
plt.xticks(linspace, [f'{m}' for m in keys], rotation=90)

# ax.set_yticks([0.25, 0.75, 0.9], minor=True)
# ax.set_yticklabels([0.25, 0.75, 0.9], minor=True)
plt.grid(which='both', axis='y', alpha=0.7, zorder=1)
plt.ylabel('Accuracy')
plt.title('Accuracy score of the selected classifiers')
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, f'eval_accuracy.pdf'))
plt.show()

In [ ]:
fig, ax = plt.subplots()

# score_val = accuracy_scores.values()
# keys = models.keys()
score_val = [accuracy_scores[k] for k in models.keys() if '_nosyn' not in k and '_bin' in k]
keys = [k.strip('_bin_resampled') for k in models.keys() if '_nosyn' not in k and '_bin' in k]

linspace = [x for x in range(len(keys))]
plt.bar(linspace, score_val, color=['C1' if s > 0.75 else 'C0' for s in score_val])
plt.xticks(linspace, [f'{m}' for m in keys], rotation=90)

# ax.set_yticks([0.25, 0.75, 0.9], minor=True)
# ax.set_yticklabels([0.25, 0.75, 0.9], minor=True)
plt.grid(which='both', axis='y', alpha=0.7, zorder=1)
plt.ylabel('Accuracy')
plt.title('Accuracy score of the selected binary classifiers')
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, f'eval_accuracy_bin.pdf'))
plt.show()

In [ ]:
fig, ax = plt.subplots()

# score_val = accuracy_scores.values()
# keys = models.keys()
score_val = [accuracy_scores[k] for k in models.keys() if '_nosyn' in k and '_bin' not in k]
keys = [k.replace('_bin_resampled', '') for k in models.keys() if '_nosyn' in k and '_bin' not in k]

linspace = [x for x in range(len(keys))]
plt.bar(linspace, score_val, color=['C1' if s > 0.75 else 'C0' for s in score_val])
plt.xticks(linspace, [f'{m}' for m in keys], rotation=90)

# ax.set_yticks([0.25, 0.75, 0.9], minor=True)
# ax.set_yticklabels([0.25, 0.75, 0.9], minor=True)
plt.ylim([0, 1])
plt.grid(which='both', axis='y', alpha=0.7, zorder=1)
plt.ylabel('Accuracy')
plt.title('Accuracy score of the selected classifiers')
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, f'eval_accuracy_nosyn.pdf'))
plt.show()

In [ ]:
fig, ax = plt.subplots()

# score_val = accuracy_scores.values()
# keys = models.keys()
score_val = [accuracy_scores[k] for k in models.keys() if '_nosyn' in k and '_bin' in k]
keys = [k.replace('_bin_resampled', '') for k in models.keys() if '_nosyn' in k and '_bin' in k]

linspace = [x for x in range(len(keys))]
plt.bar(linspace, score_val, color=['C1' if s > 0.75 else 'C0' for s in score_val])
plt.xticks(linspace, [f'{m}' for m in keys], rotation=90)

# ax.set_yticks([0.25, 0.75, 0.9], minor=True)
# ax.set_yticklabels([0.25, 0.75, 0.9], minor=True)
plt.grid(which='both', axis='y', alpha=0.7, zorder=1)
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.title('Accuracy score of the selected binary classifiers')
plt.tight_layout()
plt.savefig(os.path.join(fig_dir, f'eval_accuracy_nosyn_bin.pdf'))
plt.show()

### Confusion Matrixes

In [ ]:
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay

for model_type in models.keys():
    model = models[model_type]['model']
    x_test = models[model_type]['Xtest']
    y_test = models[model_type]['Ytest']
    y_pred = model.predict(x_test)
    conf_mat = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=model.classes_)
    # disp = ConfusionMatrixDisplay.from_estimator(model,
    #                                              x_test, y_test,
    #                                             #  display_labels=model.classes_,
    #                                              cmap=plt.cm.Blues)
    # disp.ax_.set_title(f'{model_type} Confusion Matrix')
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'{clean_name(model_type)} Confusion Matrix')
    plt.savefig(os.path.join(fig_dir, f'conf_mat_{model_type}.pdf'))
    plt.show()

In [ ]:
fig = plt.figure()

c_matrixes = []

for model_type in models.keys():
    model = models[model_type]['model']
    x_test = models[model_type]['Xtest']
    y_test = models[model_type]['Ytest']

    if accuracy_scores[model_type] > 0.8:
        y_pred = model.predict(x_test)
        conf_mat = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=model.classes_)
        # disp = ConfusionMatrixDisplay.from_estimator(model,
        #                                              x_test, y_test,
        #                                             #  display_labels=model.classes_,
        #                                              cmap=plt.cm.Blues)
        # disp.ax_.set_title(f'{model_type} Confusion Matrix')
        c_matrixes.append(disp)
        # disp.plot(cmap=plt.cm.Blues)

for i, cm in enumerate(c_matrixes):
    plt.subplot(len(c_matrixes), 2, i+1)
    disp.plot(cmap=plt.cm.Blues)
plt.title(f'Best Confusion Matrix')
plt.savefig(os.path.join(fig_dir, f'conf_mat_best.pdf'))
plt.show()

## Feature importance

Let us extract the most important features from the RFC and the Vectorizer.

In [ ]:
pipeline = models['DecisionTree_8020']['model'].best_estimator_
vectorizer = pipeline.steps[0][1]
decision_tree = pipeline.steps[1][1]
# Get indeces of sorted importance values, then the sorted feature names
sorted_idx = (-decision_tree.feature_importances_).argsort()
features_names = vectorizer.get_feature_names_out()[sorted_idx]
features_vals = decision_tree.feature_importances_[sorted_idx]
# Logging
table_data = []
for elem in zip(features_names, features_vals):
    table_data.append(elem)
feature_importance = pd.DataFrame(table_data, columns=['Feature Name', 'Importance'])
feature_importance

Sklearn default way of computing the feature importance is to measure, at training time, how well each feature decreases the impurity of the split. The measurement is then averaged across the training steps and averaged again across the ensembles, such that a single importance score per feature is obtained [1].

Another possible method for estimating features importance is to utilize _permutations of features_ [2]: intuitively, given a fitted model with a known performance score, each feature, _i.e._ column, is randomly swapped and the score of the classifier is computed. The average drop in perfomance can then be used as a score for feature importance. In fact, the idea is that features with the highest performance drop will account the most and therefore have the highest importance score.

References:

* [1] Random Forest Feature Importance Computed in 3 Ways with Python, link: https://mljar.com/blog/feature-importance-in-random-forest/
* [2] Permutation feature importance, link: https://scikit-learn.org/stable/modules/permutation_importance.html#permutation-importance

### Plotting all Feature Importance

In [ ]:
for model_type in models.keys():
    if 'DecisionTree' in model_type or 'RandomForest' in model_type or 'GradientBoosting' in model_type:
        pipeline = models[model_type]['model'].best_estimator_
        vectorizer = pipeline.steps[0][1]
        decision_tree = pipeline.steps[1][1]
        # Get indeces of sorted importance values, then the sorted feature names
        sorted_idx = (-decision_tree.feature_importances_).argsort()
        features_names = vectorizer.get_feature_names_out()[sorted_idx]
        features_vals = decision_tree.feature_importances_[sorted_idx]
        # Logging
        table_data = []
        for elem in zip(features_names, features_vals):
            table_data.append(elem)
        feature_importance = pd.DataFrame(table_data, columns=['Feature Name', 'Importance'])
        # Plotting
        ax = sns.barplot(x='Importance', y='Feature Name', data=feature_importance, orient='h', color='C0')
        plt.title(f'Feature Importance for {clean_name(model_type)}')
        plt.grid('both', axis='x', alpha=0.7)
        plt.savefig(os.path.join(fig_dir, f'feature_importance_{model_type}.pdf'), bbox_inches='tight')
        plt.show()

### SHAP Explainer

In [ ]:
# !pip install shap

In [ ]:
# import shap

# for model_type in models.keys():
#     if 'DecisionTree' in model_type or 'RandomForest' in model_type or 'GradientBoosting' in model_type:
#         pipeline = models[model_type]['model'].best_estimator_
#         vectorizer = pipeline.steps[0][1]
#         classifier = pipeline.steps[1][1]

#         X = vectorizer.fit_transform(models[model_type]['Xtest'])
#         explainer = shap.TreeExplainer(classifier, X) #, feature_names=performance_features + synthesis_features)
#         print(explainer)
#         shap_values = explainer(X)
#         # print(shap_values.data)
#         print(shap_values[0].shape)


#         idx = 3
#         exp = shap.Explanation(shap_values.values,
#                                shap_values.base_values[0][0],
#                                data=shap_values.data,
#                             #    feature_names=performance_features + synthesis_features
#                                )
#         print(exp[idx].shape)
#         shap.plots.waterfall(exp[idx])

#         # shap.force_plot(explainer.expected_value, shap_values, X)
#         # shap.summary_plot(shape_values, X)

#         # Visualize the first prediction's explanation
#         # shap.plots.waterfall(shap_values)
#         plt.show()
#         break

## Deep Learning (More training)

In [ ]:
from __future__ import print_function
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import Input

In [ ]:
# x_train = datasets['_6535_nosyn']['Xtrain']
# y_train = datasets['_6535_nosyn']['Ytrain']
# x_test = datasets['_6535_nosyn']['Xtest']
# y_test = datasets['_6535_nosyn']['Ytest']

# df = {'Train': {}, 'Test': {}}

# unique, train_counts = np.unique(y_train, return_counts=True)
# unique, test_counts = np.unique(y_test, return_counts=True)
# df['Train'] = train_counts
# df['Test'] = test_counts
# df = pd.DataFrame(df)
# df

# for c in hth_classes:
#     df['Train'][c] = np.count(y_train.astype(np.int32)
# df = pd.DataFrame({'Train': y_train.astype(np.int32),
#                    'Test': y_train.astype(np.int32)
#                    })
# df

# bar1 = sns.barplot(x='Train',  y=df['Train'], data=df, color='darkblue')
# bar2 = sns.barplot(x='Test', y=df['Test'], data=df, color='lightblue')

# plt.legend()
# plt.grid('both')
# plt.show()

In [ ]:
# x_train = datasets['_6535_nosyn']['Xtrain'].to_dict('records')
# y_train = datasets['_6535_nosyn']['Ytrain']
# x_test = datasets['_6535_nosyn']['Xtest'].to_dict('records')
# y_test = datasets['_6535_nosyn']['Ytest']

df = get_dataset(triggered, not_triggered)
# df = get_dataset(triggered, not_triggered).drop([x for x in synthesis_features if x not in ['power', 'Temp']], axis=1)
# df = get_dataset(triggered, not_triggered).drop(synthesis_features, axis=1)
x_train, y_train, x_test, y_test = split_dataset(df, train_perc=0.80)

x_train = x_train.to_dict('records')
x_test = x_test.to_dict('records')
y_train = preprocess_labels(y_train)
y_test = preprocess_labels(y_test)

unique, train_counts = np.unique(y_train, return_counts=True)
unique, test_counts = np.unique(y_test, return_counts=True)
print(pd.DataFrame({'Train': train_counts, 'Test': test_counts}))
print('')

# Vectorize features
vect = DictVectorizer(sparse=False)
vect.fit(x_train)
x_train = vect.fit_transform(x_train)
x_test = vect.fit_transform(x_test)
# Standardize Features
norm = StandardScaler()
norm.fit(x_train)
x_train = norm.fit_transform(x_train)
x_test = norm.fit_transform(x_test)
# Convert labels strings to integers
for k in hth_dict.keys():
    y_train[y_train == hth_dict[k]] = int(k)
    y_test[y_test == hth_dict[k]] = int(k)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
# Convert to one-hot encoding
onehot_enc = keras.layers.CategoryEncoding(num_tokens=len(hth_classes), output_mode='one_hot')
y_test = onehot_enc(y_test)
y_train = onehot_enc(y_train)
print(x_train.shape)
print(x_test.shape)
print(f'train mean: {np.mean(x_train):.4f}')
print(f'train std: {np.std(x_train):.4f}')
print(f'test mean: {np.mean(x_test):.4f}')
print(f'test std: {np.std(x_test):.4f}')

In [ ]:
history = None
history_prev = None
num_epochs_prev = None

In [ ]:
l2 = tf.keras.regularizers.l2(0.001)
num_epochs = 200
batch_size = 32
loss = 'categorical_crossentropy'
# opt = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.8, beta_2=0.99)
opt = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9)
# opt = tf.keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.9)

deep_model = keras.Sequential([
    keras.layers.Input(shape=(x_train.shape[-1])),
    keras.layers.Dense(32, activation='relu'), #, kernel_regularizer=l2),
    keras.layers.BatchNormalization(),
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu'), #, kernel_regularizer=l2),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation='relu'), #, kernel_regularizer=l2),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(len(hth_classes), activation='softmax'),
])

class HTHclassifier(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.dense1 = keras.layers.Dense(4, activation='relu')
    self.dense2 = keras.layers.Dense(5, activation='softmax')
    self.detector = keras.Sequential([
        keras.layers.Input(shape=(x_train.shape[-1])),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        # keras.layers.Dropout(0.5),
        # keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(1, activation='sigmoid'),
    ])
    self.classifier = keras.Sequential([
        keras.layers.Input(shape=(x_train.shape[-1])),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        # keras.layers.Dropout(0.5),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(len(hth_classes) - 1, activation='softmax'),
    ])

  def call(self, inputs):
    is_trojan = self.detector(inputs)
    hth_class = self.classifier(inputs)
    return self.dense2(x)


deep_model.compile(loss=loss, optimizer=opt, metrics='accuracy')
deep_model.summary()

In [ ]:
early_stop_loss = keras.callbacks.EarlyStopping(monitor='loss', patience=10)
early_stop_val_acc = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, mode='max', min_delta=0.05, verbose=1)
callbacks = None # [early_stop_val_acc]

history = deep_model.fit(x_train, y_train,
               epochs=num_epochs,
               batch_size=batch_size,
               validation_data=(x_test, y_test),
               callbacks=callbacks,
               verbose=1)

In [ ]:
for metric in history.history.keys():
    if 'loss' in metric:
        x = range(len(history.history['loss']))
        y = history.history[metric]
        plt.plot(x, y, label=metric)
        if history_prev is not None:
            x = range(len(history_prev.history['loss']))
            y = history_prev.history[metric]
            plt.plot(x, y, '--', label=f'Previous {metric}', alpha=0.6)
plt.grid('both')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.show()

for metric in history.history.keys():
    if 'accuracy' in metric:
        x = range(len(history.history['accuracy']))
        y = history.history[metric]
        plt.plot(x, y, label=metric)
        if history_prev is not None:
            x = range(len(history_prev.history['accuracy']))
            y = history_prev.history[metric]
            plt.plot(x, y, '--', label=f'Previous {metric}', alpha=0.6)
plt.grid('both')
plt.legend()
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.show()

history_prev = history
num_epochs_prev = num_epochs

In [ ]:
l, a = deep_model.evaluate(x_train, y_train, verbose=0)
print(f'Train accuracy: {a * 100:.2f}%')
print(f'Train loss:     {l:.4f}')

l, a = deep_model.evaluate(x_test, y_test, verbose=0)
print(f'Evaluation accuracy: {a * 100:.2f}%')
print(f'Evaluation loss:     {l:.4f}')

### Self-Supervised Learning (TabNet)

[TabNet Colab example.](https://colab.research.google.com/drive/1P8Obe07DP3VeOld08ThyT1HnChLip_LO)

## Converting Notebook to PDF

The following two cells can be ignored for grading, as they just convert this notebook into a PDF file.

In [ ]:
%%capture
!apt-get update
!apt-get install -y texlive-xetex texlive-fonts-recommended texlive-plain-generic
!apt-get install -y inkscape
!add-apt-repository -y universe
!add-apt-repository -y ppa:inkscape.dev/stable
!apt-get update -y
!apt install -y inkscape

In [ ]:
%%capture
import re

ASSIGNMENT_NAME = 'HTH_Detection'
pdf_dir = os.path.join(os.path.abspath(''), 'drive', 'MyDrive')
pdf_dir = os.path.join(pdf_dir, 'Colab Notebooks', 'hth_detection')
pdf_filename = re.escape(os.path.join(pdf_dir, ASSIGNMENT_NAME)) + '.ipynb'

!jupyter nbconvert --to pdf --TemplateExporter.exclude_input=False $pdf_filename